# Update dataset

In [1]:
import requests
import polars as pl
import math

Before March 2025 we could use this url format to scrap uniques:<br>
```
faction = 'ax'
name = 'Ada Lovelace' # 'Heimdall'
url = f'https://api.altered.gg/cards?page=1&factions%5B%5D={faction}&rarity%5B%5D=UNIQUE&translations.name="{name}"'
```
and we had MAIN_EFFECT accessible for their directly.
Since, MAIN_EFFECT disapeared from this request, and we have to use ```https://api.altered.gg/cards/ALT_COREKS_B_OR_05_U_3798``` individually

In [2]:
dataset = pl.read_parquet("dataset.parquet")

## Tests

In [3]:
faction = 'ax'
name = 'Ada Lovelace' # 'Heimdall'
url = f'https://api.altered.gg/cards?page=1&factions%5B%5D={faction}&rarity%5B%5D=UNIQUE&translations.name="{name}"'
headers = {'Accept-Language': 'en-en'}
response = requests.get(url, headers=headers, verify=False)
resp_dict = response.json()
resp_dict

c:\Users\jbonnet\Documents\python\projects\_small_ones\altered_tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.altered.gg'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'@context': '/contexts/Card',
 '@id': '/cards',
 '@type': 'hydra:Collection',
 'hydra:totalItems': 1000,
 'hydra:member': [{'id': '2BMA17XSY59FSS0BJ25JZ2Q48B',
   'cardType': {'@id': '/card_types/01H19NWA92A4ERAC4ATMSZNASS',
    '@type': 'CardType',
    'reference': 'CHARACTER',
    'id': '01H19NWA92A4ERAC4ATMSZNASS',
    'name': 'Character'},
   'cardSet': {'@id': '/card_sets/CORE',
    '@type': 'CardSet',
    'id': '01HKAFJN3HG3TWKYV0E014K01G',
    'reference': 'CORE',
    'name': 'Beyond the Gates'},
   'cardSubTypes': [],
   'rarity': {'@id': '/rarities/UNIQUE',
    '@type': 'Rarity',
    'reference': 'UNIQUE',
    'id': '01GE7AC9X35JXEQZJBBD6E4BKW',
    'name': 'Unique'},
   'imagePath': 'https://altered-prod-eu.s3.amazonaws.com/Art/CORE/CARDS/ALT_CORE_B_AX_13/UNIQUE/JPG/en_US/6e4ffd2ebc58479dfe6040f6516b2226.jpg',
   'assets': {'WEB': ['https://altered-prod-eu.s3.amazonaws.com/Art/CORE/CARDS/ALT_CORE_B_AX_13/ALT_CORE_B_AX_13_C_WEB.jpg',
     'https://altered-prod-eu.s3.amazonaws

In [4]:
n_pages_to_proceed = math.ceil(resp_dict['hydra:totalItems'] / len(resp_dict['hydra:member']))
print(f"{name} {resp_dict['hydra:totalItems']} items ({len(resp_dict['hydra:member'])} by page) {n_pages_to_proceed} pages to scrap")

Ada Lovelace 1000 items (36 by page) 28 pages to scrap


In [7]:
id_lst = [i['reference'] for i in resp_dict['hydra:member']]
id_lst[:4]

['ALT_CORE_B_AX_13_U_1000',
 'ALT_CORE_B_AX_13_U_1010',
 'ALT_CORE_B_AX_13_U_1011',
 'ALT_CORE_B_AX_13_U_1015']

In [8]:
# Convert the 'id' column of the dataset to a set for faster lookup
dataset_ids = set(dataset['id'].to_list())

# Find IDs in id_lst that are not in the dataset
missing_ids = [id_ for id_ in id_lst if id_ not in dataset_ids]

# Print the missing IDs
print(f"IDs in id_lst that are not in the dataset: {missing_ids}")

IDs in id_lst that are not in the dataset: ['ALT_CORE_B_AX_13_U_1017', 'ALT_CORE_B_AX_13_U_1029']


In [ ]:
# card_reference = missing_ids[0]
card_reference = 'ALT_CORE_B_AX_13_U_1014'  # test one that is not in the list
url = f'https://api.altered.gg/cards/{card_reference}'
headers = {'Accept-Language': 'en-en'}
response = requests.get(url, headers=headers)
resp_dict = response.json()
resp_dict

c:\Users\jbonnet\Documents\python\projects\_small_ones\altered_tools\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.altered.gg'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'code': 401, 'message': 'JWT Token not found'}

: 